https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html

In [1]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
print("PyTorch Version: ", torch.__version__)
print("Torchvision Version: ", torchvision.__version__)

PyTorch Version:  1.1.0
Torchvision Version:  0.3.0


In [2]:
#!mkdir ./data
#!wget https://download.pytorch.org/tutorial/hymenoptera_data.zip 
#!mv ./hymenoptera_data.zip ./data/
#!ls ./data/
#!apt install -y unzip
#!unzip ./data/hymenoptera_data.zip -d ./data/
!ls ./data/hymenoptera_data

train  val


In [3]:
#Inputs
data_dir = "./data/hymenoptera_data"
model_name = "squeezenet"
num_classes = 2
batch_size = 8
num_epochs = 15
feature_extract = True

In [4]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25,
               is_inception=False):
    since = time.time()
    val_acc_history = []
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print('Epoch {} / {}'.format(epoch, num_epochs, - 1))
        print('-' * 10)
        
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train() #Set model to training mode
            else:
                model.eval()  #Set model to evaluate mode
            
            running_loss = 0.0
            running_corrects = 0
            
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                #zero the parameter gradients
                optimizer.zero_grad()
                
                #forward
                #track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    #Get model outputs and calculate loss
                    #Special case for inception because in training it has an auxiliary output in train.
                    #mode we calculate the loss by summarizing the final output the auxiliary output
                    #but in testing we only consider the final output
                    
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                        
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)
                        
                    _, preds = torch.max(outputs, 1)
                    
                    #backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        
                #statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)
            
            print('{} loss: {:.4f} Acc {:.4f}'.format(phase, epoch_loss, epoch_acc))
            
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)
                
        print()
        
    time_elapsed = time.time() - since()
    print('Training complete in {:.0f}m {:.0f}s '.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    
    model.load_state_dict(best_model_wts)
    
    return model, val_acc_history
                        

In [5]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [6]:
#Note: inception_v3 requires the inputsize to be (299, 299), whereas all of the other models expect (224, 224)


In [7]:
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    #Initialize these variables whichwill be set in this if statement, each of these variables is model specific
    
    model_ft = None
    
    input_size = 0
    
    if model_name == "resnet":
        #ResNet18
        model_ft = models.resnet18(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc =nn.Linear(num_ftrs, num_classes)
        input_size = 224
        
    elif model_name == "alexnet":
        #Alexnet
        model_ft= models.alexnt(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs, num_classes)
        input_size = 224
    elif model_name == "vgg":
        #VGG11_bn
        model_ft = models.vgg11_bn(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs, num_classes)
        input_size = 224
    elif model_name == "squeezenet":
        #Squeezenet
        model_ft = models.squeezenet1_0(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        model_ft.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
        model_ft.num_classes = num_classes
        input_size = 224
    elif model_name == "densenet":
        #Densenet
        model_ft = models.densenet121(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes)
        input_size = 224
    elif model_name == "inception":
        #Inception v3
        #Becareful, expects (299, 299) sized images and has auxiliary output
        model_ft = models.inception_v3(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        #Handle the auxilary net
        num_ftrs = model_ft.AuxiLogits.fc.in_features
        model_ft.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)
        #Handle the primary net
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 299
        
    else:
        print("Invalid model name, exiting...")
        exit()
        
    return model_ft, input_size

    

In [8]:
#Initialize the model for this run
model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)
print(model_ft)

SqueezeNet(
  (features): Sequential(
    (0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (3): Fire(
      (squeeze): Conv2d(96, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace)
    )
    (4): Fire(
      (squeeze): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace)
    )
    (5): Fire(
      (squeeze): Conv2d(128, 32, kerne

In [9]:
#Load data

#Data augumentation and normalization for training
#Just normalization for validation

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([transforms.Resize(input_size),
                              transforms.CenterCrop(input_size),
                              transforms.ToTensor(),
                              transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                              ]),
}

print("Intializing Datasets and Dataloaders...")

#creating training and validation datasets
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                         data_transforms[x]) for x in ['train', 'val']}
#Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x],
                                                  batch_size=batch_size, shuffle=True, num_workers=4) for x in ['train', 'val']}
#Detect if we havea GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Intializing Datasets and Dataloaders...


In [10]:
model_ft = model_ft.to(device)

params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name, param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t", name)
else:
    for name, param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t", name)
            
#Observe that all parameters are being optimized
optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

Params to learn:
	 classifier.1.weight
	 classifier.1.bias


In [ ]:
#Run Trai ning and Validation Step
criterion = nn.CrossEntropyLoss()

#Train and evaluate
model_ft, hist = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, 
                            num_epochs=num_epochs, is_inception=(model_name=="inception"))

Epoch 0 / 15
----------
train loss: 0.4856 Acc 0.7582
val loss: 0.3971 Acc 0.8431

Epoch 1 / 15
----------
train loss: 0.2766 Acc 0.8811
val loss: 0.3477 Acc 0.8954

Epoch 2 / 15
----------
train loss: 0.2111 Acc 0.9016
val loss: 0.3876 Acc 0.8889

Epoch 3 / 15
----------
train loss: 0.2509 Acc 0.9057
val loss: 0.3362 Acc 0.8889

Epoch 4 / 15
----------
train loss: 0.1966 Acc 0.9262
val loss: 0.3389 Acc 0.9085

Epoch 5 / 15
----------
train loss: 0.1820 Acc 0.9303
val loss: 0.3445 Acc 0.9085

Epoch 6 / 15
----------
train loss: 0.1788 Acc 0.9262
val loss: 0.3713 Acc 0.9150

Epoch 7 / 15
----------
train loss: 0.1703 Acc 0.9180
val loss: 0.3892 Acc 0.9150

Epoch 8 / 15
----------
train loss: 0.1619 Acc 0.9385
val loss: 0.3633 Acc 0.9150

Epoch 9 / 15
----------
train loss: 0.1494 Acc 0.9262
val loss: 0.3419 Acc 0.9150

Epoch 10 / 15
----------
train loss: 0.1520 Acc 0.9426
val loss: 0.3271 Acc 0.9216

Epoch 11 / 15
----------
train loss: 0.1572 Acc 0.9344
val loss: 0.3230 Acc 0.9216

Ep